# Import Packages

In [1]:
from packages import *
%matplotlib inline

# Read Dataset

In [2]:
all_files = glob.glob('virufy_clean/*')
l1, l2 = [], []
for i in all_files: 
    l1.append(i)
    label = i.split('\\')[1][:3].strip()
    l2.append(label)
df = pd.DataFrame(zip(l1,l2),columns=['fname','label'])
df.head()

,fname,label
0,virufy_clean\neg-0421-083-cough-m-53-1.wav,neg
1,virufy_clean\neg-0421-083-cough-m-53-10.wav,neg
2,virufy_clean\neg-0421-083-cough-m-53-11.wav,neg
3,virufy_clean\neg-0421-083-cough-m-53-12.wav,neg
4,virufy_clean\neg-0421-083-cough-m-53-13.wav,neg


# Get class distribution

In [3]:
df.set_index('fname', inplace=True)

for f in df.index:
    signal,rate = librosa.load(f)
    df.at[f, 'length'] = signal.shape[0]/rate

df.head()

,label,length
fname,,
virufy_clean\neg-0421-083-cough-m-53-1.wav,neg,0.473469
virufy_clean\neg-0421-083-cough-m-53-10.wav,neg,0.438277
virufy_clean\neg-0421-083-cough-m-53-11.wav,neg,0.511202
virufy_clean\neg-0421-083-cough-m-53-12.wav,neg,0.462132
virufy_clean\neg-0421-083-cough-m-53-13.wav,neg,0.537914


In [4]:
# get class names
classes = list(np.unique(df.label))

# get probability distribution of classes based on duration of audio files
class_dist = df.groupby(['label'])['length'].mean()
prob_dist = class_dist/class_dist.sum()

# Get 2 * audio samples of duration 1/10 secs taken from all audio files
n_samples = int(2 * (1/0.1)* df['length'].sum())
n_samples

1494

# Feature Extraction

In [5]:
def build_random_feat():
    
    X = []
    y = []
    
    for _ in range(n_samples):
        
        # choose a random audio sample from dataset according to their probability distribution
        rand_class = np.random.choice(class_dist.index,p=prob_dist)
        file = np.random.choice(df[df.label==rand_class].index)
        rate, wav = wavfile.read(file)
        label = df.at[file,'label']
        
        # crop a random duration of 1/10 sec from chosen audio sample
        step=int(rate/10)
        rand_index = np.random.randint(0,wav.shape[0]-step)
        sample = wav[rand_index:rand_index+step]
        
        # get stat features of the cropped duration
        mt, st, mt_n = aF.mid_feature_extraction(sample, rate, 
                                                 mid_window = 1 * rate, 
                                                 mid_step = 1 * rate, 
                                                 short_window = 0.025 * rate, # 25 ms frame (n_fft)
                                                 short_step = 0.010 * rate) # 10 ms stride (hop length)
        mt = np.transpose(mt)
        
        for i in mt:
            X.append(list(i))
            y.append(classes.index(label))
            
    df2 = pd.DataFrame(X, columns = mt_n)
    df2['label'] = y
                   
    return df2

In [6]:
df = build_random_feat()
df.head()

,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,spectral_flux_mean,spectral_rolloff_mean,mfcc_1_mean,mfcc_2_mean,...,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std,label
0,0.230263,0.053093,3.096664,0.257470,0.191442,1.717253,0.004522,0.352500,-23.774168,0.734928,...,0.001975,0.000541,0.002048,0.001278,0.004800,0.024604,0.017330,0.001485,0.004359,0
1,0.150689,0.045410,2.966139,0.218944,0.201535,1.432761,0.004453,0.245000,-23.879562,1.501665,...,0.010020,0.009123,0.012284,0.000953,0.005942,0.015724,0.014974,0.001648,0.003425,0
2,0.185464,0.026669,3.158287,0.238257,0.191323,1.818111,0.003814,0.288750,-23.581316,0.776733,...,0.016781,0.005842,0.012063,0.001272,0.009529,0.004164,0.004134,0.001803,0.002694,1
3,0.339912,0.075237,3.235773,0.376918,0.234236,2.489605,0.002372,0.535000,-22.530058,0.678329,...,0.005523,0.002703,0.006831,0.002548,0.002626,0.006559,0.004813,0.002747,0.001440,1
4,0.090539,0.036343,3.092085,0.175022,0.200372,0.934282,0.007286,0.123125,-24.629810,2.810257,...,0.016250,0.000777,0.025196,0.000464,0.001000,0.010290,0.019469,0.000393,0.003233,1


In [7]:
df.shape

(1494, 137)

In [9]:
l = ['zcr','energy','energy_entropy']
temporal = []
fourier = []
for nam in df.columns[:-1]:
    
    flag = False
    
    for i in l:
        if(i in nam):
            flag = True
            break
    
    if(flag==True):  
        temporal.append(nam)
    else:
        fourier.append(nam)

In [10]:
temporal

['zcr_mean',
 'energy_mean',
 'energy_entropy_mean',
 'delta zcr_mean',
 'delta energy_mean',
 'delta energy_entropy_mean',
 'zcr_std',
 'energy_std',
 'energy_entropy_std',
 'delta zcr_std',
 'delta energy_std',
 'delta energy_entropy_std']

In [11]:
fourier

['spectral_centroid_mean',
 'spectral_spread_mean',
 'spectral_entropy_mean',
 'spectral_flux_mean',
 'spectral_rolloff_mean',
 'mfcc_1_mean',
 'mfcc_2_mean',
 'mfcc_3_mean',
 'mfcc_4_mean',
 'mfcc_5_mean',
 'mfcc_6_mean',
 'mfcc_7_mean',
 'mfcc_8_mean',
 'mfcc_9_mean',
 'mfcc_10_mean',
 'mfcc_11_mean',
 'mfcc_12_mean',
 'mfcc_13_mean',
 'chroma_1_mean',
 'chroma_2_mean',
 'chroma_3_mean',
 'chroma_4_mean',
 'chroma_5_mean',
 'chroma_6_mean',
 'chroma_7_mean',
 'chroma_8_mean',
 'chroma_9_mean',
 'chroma_10_mean',
 'chroma_11_mean',
 'chroma_12_mean',
 'chroma_std_mean',
 'delta spectral_centroid_mean',
 'delta spectral_spread_mean',
 'delta spectral_entropy_mean',
 'delta spectral_flux_mean',
 'delta spectral_rolloff_mean',
 'delta mfcc_1_mean',
 'delta mfcc_2_mean',
 'delta mfcc_3_mean',
 'delta mfcc_4_mean',
 'delta mfcc_5_mean',
 'delta mfcc_6_mean',
 'delta mfcc_7_mean',
 'delta mfcc_8_mean',
 'delta mfcc_9_mean',
 'delta mfcc_10_mean',
 'delta mfcc_11_mean',
 'delta mfcc_12_mean'

# Save data

In [8]:
df.to_csv('data/TempFour_features.csv',index=False)